In [1]:
import os 
import face_recognition
import matplotlib.pyplot as plt
import cv2
import pickle

# Validating Images

**It is better to validate the images to see if face is detected or not (only one face should be detected) in every face angle before generating encodings. Afer validations, we can generate encoding.**

**Note: There should be 'to_encode_faces' folder in the directory where this notebook (validateData_generateEncodings.ipynb) is located. All the images to encode should be inside 'to_encode_faces' folder in following format:**

```
to_encode/
├── <person_1_id>-<university>-<first_name>-<last_name>/
│   ├── front.jpg
│   ├── down.jpg
│   ├── up.jpg
│   ├── left.jpg
│   └── right.jpg
├── <person_2_id>-<university>-<first_name>-<last_name>/
│   ├── front.jpg
│   ├── down.jpg
│   ├── up.jpg
│   ├── left.jpg
│   └── right.jpg
├── <person_3_id>-<university>-<first_name>-<last_name>/
│   ├── front.jpg
│   ├── down.jpg
│   ├── up.jpg
│   ├── left.jpg
│   └── right.jpg
└── ...

Don't keep < or > in the folder name! It is used to denote a variable which can have different values. 
The folder should be named like: 23140736-BCU-ABHASH-RAI
```

In [3]:
def validate_images(images_to_encode_folder_path):

    to_recapture = []

    all_directories = os.listdir(images_to_encode_folder_path)

    for folder in all_directories:

        all_files = os.listdir(f'{images_to_encode_folder_path}/{folder}')
        img_path = [filename for filename in all_files if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.gif', '.bmp'))] # Filter for image files (e.g., .jpg, .png, .jpeg)
        
        for img in img_path:
            
            image = cv2.imread(f'{images_to_encode_folder_path}/{folder}/{img}')
            
            # Detect faces in the image
            face_locations = face_recognition.face_locations(image)

            current_img_path = f'{images_to_encode_folder_path}/{folder}/{img}'
            
            if len(face_locations) == 0:
                print(f'* "{current_img_path}" - No Faces Detected! Bad!')
                to_recapture.append(current_img_path)
            elif len(face_locations) > 1:
                print(f'* "{current_img_path}" - More than one Faces Detected! Bad!')
                to_recapture.append(current_img_path)
            elif len(face_locations) == 1:
                print(f'"{current_img_path}" - Single face detected. Good.')

    if len(to_recapture) != 0:           
        print(f'\n\nThe below images either have multiple faces or cannot detect a face at all. Recapture the image of people given below to have detectable single face:')
        print(to_recapture)
    else:
        print('\n\nCan detect a single face in every image. All good. Proceed to generating encodings.')

validate_images("./to_encode_faces")

"./to_encode_faces/23140736-BCU-ABHASH-RAI/down.jpg" - Single face detected. Good.
"./to_encode_faces/23140736-BCU-ABHASH-RAI/front.jpg" - Single face detected. Good.
"./to_encode_faces/23140736-BCU-ABHASH-RAI/left.jpg" - Single face detected. Good.
"./to_encode_faces/23140736-BCU-ABHASH-RAI/right.jpg" - Single face detected. Good.
"./to_encode_faces/23140736-BCU-ABHASH-RAI/up.jpg" - Single face detected. Good.
"./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/down.jpg" - Single face detected. Good.
"./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/front.jpg" - Single face detected. Good.
"./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/left.jpg" - Single face detected. Good.
"./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/right.jpg" - Single face detected. Good.
"./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/up.jpg" - Single face detected. Good.
"./to_encode_faces/23140745-BCU-SADIKSHYA-GHIMIRE/down.jpg" - Single face detected. Good.
"./to_encode_faces/23140745-BCU-SADIKSHYA-GHIMIRE/front.jpg"

# Generate Face Encodings

In [4]:

def generate_encodings(images_to_encode_folder_path, path_to_store_encodings):

    all_face_encodings = {}

    all_directories = os.listdir(images_to_encode_folder_path)

    for folder in all_directories:
        
        face_encodings = []

        all_files = os.listdir(f'{images_to_encode_folder_path}/{folder}')
        img_path = [filename for filename in all_files if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.gif', '.bmp'))] # Filter for image files (e.g., .jpg, .png, .jpeg)
        
        for img in img_path:
            image = face_recognition.api.load_image_file(f'{images_to_encode_folder_path}/{folder}/{img}')
            encoding = face_recognition.api.face_encodings(image, model='large')
            if len(encoding) == 1:
                face_encodings.append(encoding[0])
                print(f'Successfully generated face encodings of "{images_to_encode_folder_path}/{folder}/{img}"')
            elif len(encoding) == 0:
                print(f'No face detected in {images_to_encode_folder_path}/{folder}/{img}')
                return
            elif len(encoding) > 1:
                print(f'More than one faces detected in {images_to_encode_folder_path}/{folder}/{img}')
                return
            
        all_face_encodings[folder] = face_encodings
        
    with open(path_to_store_encodings, 'wb') as file:
        pickle.dump(all_face_encodings, file)

generate_encodings(
    images_to_encode_folder_path = "./to_encode_faces",
    path_to_store_encodings = './encodings/multiple_angles_faces_encodings'
)

Successfully generated face encodings of "./photos/23140736-BCU-ABHASH-RAI/down.jpg"
Successfully generated face encodings of "./photos/23140736-BCU-ABHASH-RAI/front.jpg"
Successfully generated face encodings of "./photos/23140736-BCU-ABHASH-RAI/left.jpg"
Successfully generated face encodings of "./photos/23140736-BCU-ABHASH-RAI/right.jpg"
Successfully generated face encodings of "./photos/23140736-BCU-ABHASH-RAI/up.jpg"
Successfully generated face encodings of "./photos/23140741-BCU-NANAK-SHRESTHA/down.jpg"
Successfully generated face encodings of "./photos/23140741-BCU-NANAK-SHRESTHA/front.jpg"
Successfully generated face encodings of "./photos/23140741-BCU-NANAK-SHRESTHA/left.jpg"
Successfully generated face encodings of "./photos/23140741-BCU-NANAK-SHRESTHA/right.jpg"
Successfully generated face encodings of "./photos/23140741-BCU-NANAK-SHRESTHA/up.jpg"
Successfully generated face encodings of "./photos/23140745-BCU-SADIKSHYA-GHIMIRE/down.jpg"
Successfully generated face encodings 

In [4]:

def generate_encodings(images_to_encode_folder_path, path_to_store_encodings):

    all_face_encodings = {}

    all_directories = os.listdir(images_to_encode_folder_path)

    for folder in all_directories:
        
        face_encodings = []

        all_files = os.listdir(f'{images_to_encode_folder_path}/{folder}')
        img_path = [filename for filename in all_files if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.gif', '.bmp'))] # Filter for image files (e.g., .jpg, .png, .jpeg)
        
        for img in img_path:
            image = face_recognition.api.load_image_file(f'{images_to_encode_folder_path}/{folder}/{img}')
            encoding = face_recognition.api.face_encodings(image, model='large')
            if len(encoding) == 1:
                face_encodings.append(encoding[0])
                print(f'Successfully generated face encodings of "{images_to_encode_folder_path}/{folder}/{img}"')
            elif len(encoding) == 0:
                print(f'No face detected in {images_to_encode_folder_path}/{folder}/{img}')
                return
            elif len(encoding) > 1:
                print(f'More than one faces detected in {images_to_encode_folder_path}/{folder}/{img}')
                return
            
        all_face_encodings[folder] = face_encodings
        
    with open(path_to_store_encodings, 'wb') as file:
        pickle.dump(all_face_encodings, file)

generate_encodings(
    images_to_encode_folder_path = "./to_encode_faces",
    path_to_store_encodings = './encodings/multiple_angles_faces_encodings'
)

Successfully generated face encodings of "./to_encode_faces/23140736-BCU-ABHASH-RAI/down.jpg"
Successfully generated face encodings of "./to_encode_faces/23140736-BCU-ABHASH-RAI/front.jpg"
Successfully generated face encodings of "./to_encode_faces/23140736-BCU-ABHASH-RAI/left.jpg"
Successfully generated face encodings of "./to_encode_faces/23140736-BCU-ABHASH-RAI/right.jpg"
Successfully generated face encodings of "./to_encode_faces/23140736-BCU-ABHASH-RAI/up.jpg"
Successfully generated face encodings of "./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/down.jpg"
Successfully generated face encodings of "./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/front.jpg"
Successfully generated face encodings of "./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/left.jpg"
Successfully generated face encodings of "./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/right.jpg"
Successfully generated face encodings of "./to_encode_faces/23140741-BCU-NANAK-SHRESTHA/up.jpg"
Successfully generated face encodings of